<a href="https://colab.research.google.com/github/tejinderpa/Deep-Learning/blob/Going-Deep/Pretrained_models_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#constructing AlexNet
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

In [ ]:
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(227, 227, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# 2nd Convolutional Layer
model.add(Conv2D(256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# 3rd Convolutional Layer
model.add(Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))

# 4th Convolutional Layer
model.add(Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))

# 5th Convolutional Layer
model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Flattening the layers
model.add(Flatten())

# 1st Fully Connected Layer
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

# 2nd Fully Connected Layer
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(1000, activation='softmax'))

## Using Pretrained Models


In [7]:
import keras
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')



102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
list = ['catimage', 'dogimage']

In [9]:
import numpy as np
for i in list:
  img_path = '/content/{}.jpeg'.format(i)
  img = keras.utils.load_img(img_path, target_size=(224, 224))
  x = keras.utils.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)

  preds = model.predict(x)
  print('Image is of:', i)
  print('Predicted:', decode_predictions(preds, top=3)[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Image is of: catimage
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Predicted: [('n02123045', 'tabby', np.float32(0.668096)), ('n02123159', 'tiger_cat', np.float32(0.177875)), ('n02124075', 'Egyptian_cat', np.float32(0.14905319))]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
Image is of: dogimage
Predicted: [('n02095889', 'Sealyham_terrier', np.float32(0.32429296)), ('n02099712', 'Labrador_retriever', np.float32(0.20462911)), ('n02104029', 'kuvasz', np.float32(0.16618325))]


In [10]:
import torch.nn as nn

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, 11, stride=4),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2),
            nn.Conv2d(96, 256, 5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2),
            nn.Conv2d(256, 384, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, 1000)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


In [11]:
import torch
from PIL import Image
from torchvision import transforms

# Assuming 'list' contains image filenames like 'catimage', 'dogimage'
# Load and preprocess the images
transform = transforms.Compose([
    transforms.Resize((227, 227)),  # AlexNet expects 227x227 input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Imagenet standards
])

# Instantiate the AlexNet model
model = AlexNet()

# Load pre-trained weights (if available, otherwise train the model)
# model.load_state_dict(torch.load('alexnet_weights.pth')) # Uncomment and provide path to weights if you have them
model.eval() # Set the model to evaluation mode

with torch.no_grad():
    for i in list:
        img_path = '/content/{}.jpeg'.format(i)
        img = Image.open(img_path).convert('RGB')
        img_tensor = transform(img)
        img_tensor = img_tensor.unsqueeze(0) # Add batch dimension

        # Make prediction
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs.data, 1)

        # Print results
        print('Image is of:', i)
        # Note: To get meaningful class names, you would need an index-to-label mapping for the model's output.
        # For now, we will just print the predicted index.
        print('Predicted class index:', predicted.item())

Image is of: catimage
Predicted class index: 754
Image is of: dogimage
Predicted class index: 754


In [12]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            # Conv layers
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # Add more layers similarly...
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 56 * 56, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 1000),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


In [15]:
import torchvision
torchvision.models.googlenet(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 64.8MB/s]


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [16]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 137MB/s]


In [17]:
import torch
from PIL import Image
from torchvision import transforms
import torchvision.models as models

# Load pre-trained GoogLeNet model
googlenet_model = models.googlenet(weights=models.GoogLeNet_Weights.IMAGENET1K_V1)
googlenet_model.eval()

# Load pre-trained ResNet18 model
resnet18_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
resnet18_model.eval()

# Image transformations for GoogLeNet and ResNet (typically the same for models trained on ImageNet)
transform_imagenet = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load ImageNet class labels
# You'll need a file containing the ImageNet class index to label mapping.
# For demonstration, we'll use a placeholder and print the index.
# In a real scenario, you would load a dictionary mapping indices to class names.
try:
    import json
    with open('/root/.cache/torch/hub/checkpoints/imagenet_class_index.json') as f:
        imagenet_labels = json.load(f)
    # Convert keys to integers
    imagenet_labels = {int(k): v[1] for k, v in imagenet_labels.items()}
except FileNotFoundError:
    print("ImageNet class index file not found. Predictions will be class indices.")
    imagenet_labels = None


# Predict using GoogLeNet
print("Predictions using GoogLeNet:")
with torch.no_grad():
    for i in list:
        img_path = '/content/{}.jpeg'.format(i)
        img = Image.open(img_path).convert('RGB')
        img_tensor = transform_imagenet(img)
        img_tensor = img_tensor.unsqueeze(0) # Add batch dimension

        outputs = googlenet_model(img_tensor)
        _, predicted = torch.max(outputs.data, 1)

        print(f'Image is of: {i}')
        if imagenet_labels:
            print(f'Predicted: {imagenet_labels[predicted.item()]}')
        else:
            print(f'Predicted class index: {predicted.item()}')

print("\nPredictions using ResNet18:")
# Predict using ResNet18
with torch.no_grad():
    for i in list:
        img_path = '/content/{}.jpeg'.format(i)
        img = Image.open(img_path).convert('RGB')
        img_tensor = transform_imagenet(img)
        img_tensor = img_tensor.unsqueeze(0) # Add batch dimension

        outputs = resnet18_model(img_tensor)
        _, predicted = torch.max(outputs.data, 1)

        print(f'Image is of: {i}')
        if imagenet_labels:
             print(f'Predicted: {imagenet_labels[predicted.item()]}')
        else:
            print(f'Predicted class index: {predicted.item()}')

ImageNet class index file not found. Predictions will be class indices.
Predictions using GoogLeNet:
Image is of: catimage
Predicted class index: 281
Image is of: dogimage
Predicted class index: 208

Predictions using ResNet18:
Image is of: catimage
Predicted class index: 281
Image is of: dogimage
Predicted class index: 190
